## **Exemplos: Multi-Agentes de IA para Reservar Hotel**

No mundo acelerado de hoje, planejar uma viagem de negócios envolve mais do que apenas reservar um voo e um quarto de hotel. Exige um nível de coordenação e eficiência que pode ser difícil de alcançar. É aqui que os Multi-Agentes de IA entram em cena, revolucionando a forma como gerenciamos nossas necessidades de viagem.

Imagine ter uma equipe de agentes inteligentes à sua disposição, trabalhando juntos para lidar com todos os aspectos da sua viagem com precisão e facilidade. Com nossa tecnologia avançada de IA, criamos agentes especializados para serviços de reserva e organização de itinerários, garantindo uma experiência de viagem tranquila e sem estresse.

Este é um cenário básico. Ao planejar uma viagem de negócios, precisamos consultar um agente de viagens corporativas para obter informações sobre passagens aéreas, hotéis, etc. Por meio de Agentes de IA, podemos criar agentes para serviços de reserva e agentes para organização de itinerários que colaboram entre si e aumentam o nível de inteligência.


# Inicializar o Serviço de Agente de IA do Azure e obter informações de configuração do **.env**

### **.env**

Crie um arquivo .env

O arquivo **.env** contém a string de conexão do Serviço de Agente de IA do Azure, o modelo usado pelo AOAI e o serviço de API de Pesquisa do Google correspondente, ENDPOINT, etc.

- **AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME** = "Nome de Implantação do Modelo do Serviço de Agente de IA do Azure"

[**NOTE**] Você precisará de um modelo com Limite de Taxa de 100.000 (Tokens por minuto) e Limite de Taxa de 600 (Requisições por minuto).

  Você pode obter o modelo no Azure AI Foundry - Model and Endpoint.

- **AZURE_AI_AGENT_PROJECT_CONNECTION_STRING** = "String de Conexão do Projeto do Serviço de Agente de IA do Azure"

  Você pode obter a string de conexão do projeto na visão geral do seu projeto na tela do Portal AI Foundry.

- **SERPAPI_SEARCH_API_KEY** = "Sua API KEY de Pesquisa do SERPAPI"
- **SERPAPI_SEARCH_ENDPOINT** = "Seu ENDPOINT de Pesquisa do SERPAPI"

Para obter o Nome de Implantação do Modelo e a String de Conexão do Projeto do Serviço de Agente de IA do Azure, você precisa criar o Serviço de Agente de IA do Azure. É recomendado usar [este template](https://portal.azure.com/#create/Microsoft.Template/uri/https%3A%2F%2Fraw.githubusercontent.com%2Ffosteramanda%2Fazure-agent-quickstart-templates%2Frefs%2Fheads%2Fmaster%2Fquickstarts%2Fmicrosoft.azure-ai-agent-service%2Fstandard-agent%2Fazuredeploy.json) para criá-lo diretamente （***Nota:*** O Serviço de Agente de IA do Azure está atualmente configurado em uma região limitada. É recomendado que você consulte [este link](https://learn.microsoft.com/en-us/azure/ai-services/agents/concepts/model-region-support) para configurar a região).

O agente precisa acessar o SERPAPI. É recomendado registrar-se usando [este link](https://serpapi.com/searches). Após o registro, você poderá obter uma API KEY e ENDPOINT únicos.


# Fazer login no Azure

Agora você precisa fazer login no Azure. Abra um terminal no VScode e execute o comando `az login`.


# Configuração

Para executar este notebook, você precisará instalar as seguintes bibliotecas. Aqui está uma lista das bibliotecas necessárias e os comandos correspondentes para instalação com pip:

azure-identity: Para autenticação no Azure.  
requests: Para realizar requisições HTTP.  
semantic-kernel: Para o framework de kernel semântico (assumindo que esta seja uma biblioteca personalizada ou específica, pode ser necessário instalá-la de uma fonte ou repositório específico).  


In [ ]:
!pip install azure-identity
!pip install requests
!pip install semantic-kernel
!pip install --upgrade semantic_kernel
!pip install azure-cli

# Explicação: 
import asyncio: Isso importa o módulo asyncio, que oferece suporte para programação assíncrona em Python. Ele permite que você escreva código concorrente usando a sintaxe async e await.  
from typing import Annotated: Isso importa o tipo Annotated do módulo typing. Annotated é usado para adicionar metadados às anotações de tipo, o que pode ser útil para diversos propósitos, como validação, documentação ou ferramentas.  


In [ ]:
import asyncio,os
from typing import Annotated

# Explicação:
Ao usar from dotenv import load_dotenv e load_dotenv(), você pode gerenciar facilmente configurações e informações sensíveis (como chaves de API e URLs de banco de dados) em um arquivo .env, mantendo-as separadas do seu código-fonte e tornando sua aplicação mais segura e fácil de configurar.


In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Explicação:

Declaração de Importação: from azure.identity.aio import DefaultAzureCredential: Isso importa a classe DefaultAzureCredential do módulo azure.identity.aio. A parte "aio" no nome do módulo indica que ele foi projetado para operações assíncronas.

Propósito do DefaultAzureCredential: A classe DefaultAzureCredential faz parte do Azure SDK para Python. Ela fornece uma maneira padrão de autenticar com os serviços do Azure. Ela tenta autenticar usando vários métodos em uma ordem específica, como variáveis de ambiente, identidade gerenciada e credenciais do Azure CLI.

Operações Assíncronas: O módulo "aio" indica que a classe DefaultAzureCredential suporta operações assíncronas. Isso significa que você pode usá-la com asyncio para realizar solicitações de autenticação sem bloqueio.


In [ ]:
from azure.identity.aio import DefaultAzureCredential

# Explicação:
Importa vários módulos e classes do pacote semantic_kernel. Aqui está um detalhamento de cada importação:

AgentGroupChat de semantic_kernel.agents: Esta classe lida com funcionalidades relacionadas ao chat em grupo para agentes de IA. AzureAIAgent e AzureAIAgentSettings de semantic_kernel.agents.azure_ai:

AzureAIAgent: Esta classe é usada para criar e gerenciar agentes de IA que utilizam os serviços de IA do Azure.

AzureAIAgentSettings: Esta classe é usada para configurar as definições do AzureAIAgent. TerminationStrategy de semantic_kernel.agents.strategies.termination.termination_strategy:

Esta classe define estratégias para encerrar a execução de agentes de IA sob certas condições. ChatMessageContent de semantic_kernel.contents.chat_message_content:

Esta classe é usada para gerenciar o conteúdo de mensagens de chat.
AuthorRole de semantic_kernel.contents.utils.author_role:

Esta classe define diferentes papéis para autores no contexto de mensagens de chat.

kernel_function de semantic_kernel.functions.kernel_function_decorator: Este decorador é usado para definir funções do kernel, que são funções que podem ser executadas dentro do framework semantic kernel.
Essas importações configuram os componentes necessários para criar e gerenciar agentes de IA que podem interagir em um ambiente de chat em grupo, possivelmente para tarefas como reserva de hotéis ou atividades similares.


In [ ]:
from semantic_kernel.agents import AgentGroupChat
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings
from semantic_kernel.agents.strategies.termination.termination_strategy import TerminationStrategy
from semantic_kernel.contents import ChatMessageContent
from semantic_kernel.contents import AuthorRole
from semantic_kernel.functions.kernel_function_decorator import kernel_function

# Explicação:
Em seguida, importamos a classe CodeInterpreterTool do módulo azure.ai.projects.models.

CodeInterpreterTool: Esta classe faz parte do SDK do Azure AI e é usada para interpretar e executar código no contexto de projetos de IA. Ela oferece funcionalidades para executar trechos de código, analisar código ou integrar a execução de código em fluxos de trabalho de IA.  
Essa importação configura o componente necessário para utilizar o CodeInterpreterTool no seu projeto, o que pode ser útil para tarefas que envolvam interpretar e executar código de forma dinâmica.


In [ ]:
from azure.ai.projects.models import CodeInterpreterTool

# Explicação: 
A classe ApprovalTerminationStrategy fornece uma estratégia específica para encerrar a operação de um agente de IA. O agente será encerrado se a última mensagem em seu histórico de interações contiver a palavra "salvo". Isso pode ser útil em cenários onde a tarefa do agente é considerada concluída assim que recebe a confirmação de que algo foi "salvo". Defina o método de interação. Após o plano de reserva ser salvo, ele pode ser interrompido ao receber o sinal de salvo.


In [ ]:
class ApprovalTerminationStrategy(TerminationStrategy):
    """A strategy for determining when an agent should terminate."""

    async def should_agent_terminate(self, agent, history):
        """Check if the agent should terminate."""
        return "saved" in history[-1].content.lower()

# Explicação:

A linha de código inicializa um objeto AzureAIAgentSettings com configurações padrão ou predefinidas ao chamar o método create(). Esse objeto de configurações (ai_agent_settings) pode então ser usado para configurar e gerenciar uma instância de AzureAIAgent.


In [ ]:
ai_agent_settings = AzureAIAgentSettings.create()

# Explicação:
Ao importar a biblioteca requests, você pode facilmente fazer requisições HTTP e interagir com serviços web no seu código Python.


In [ ]:
import requests

# Explicação:
Esta é uma variável que armazena a chave de API para acessar um serviço de API de SERP (Search Engine Results Page). Uma chave de API é um identificador único usado para autenticar solicitações associadas à sua conta.

'GOOGLE_SEARCH_API_KEY': Este é um texto de exemplo. Você precisa substituir 'GOOGLE_SEARCH_API_KEY' pela sua chave de API real do SERP.

Propósito: O propósito desta linha é armazenar a chave de API em uma variável para que ela possa ser usada para autenticar solicitações ao serviço de API de SERP. A chave de API é necessária para acessar o serviço e realizar buscas.

Como obter uma chave de API de SERP: Para obter uma chave de API de SERP, siga estas etapas gerais em https://serpapi.com (os passos exatos podem variar dependendo do serviço de API de SERP específico que você está utilizando):

Escolha um serviço de API de SERP: Existem vários serviços de API de SERP disponíveis, como SerpAPI, Google Custom Search JSON API, entre outros. Escolha o que melhor atende às suas necessidades.

Cadastre-se em uma conta:

Acesse o site do serviço de API de SERP escolhido https://www.serpapi.com e cadastre-se em uma conta. Pode ser necessário fornecer algumas informações básicas e verificar seu endereço de e-mail.

Crie uma chave de API:

Após se cadastrar, faça login na sua conta e navegue até a seção de API ou painel de controle. Procure uma opção para criar ou gerar uma nova chave de API.
Copie a chave de API:

Depois que a chave de API for gerada, copie-a. Essa chave será usada para autenticar suas solicitações ao serviço de API de SERP.
Substitua o texto de exemplo:

Substitua o texto de exemplo no seu arquivo .env


In [ ]:
SERPAPI_SEARCH_API_KEY=os.getenv('SERPAPI_SEARCH_API_KEY')

In [ ]:
SERPAPI_SEARCH_ENDPOINT = os.getenv('SERPAPI_SEARCH_ENDPOINT')

# Explicação:
A classe BookingPlugin fornece métodos para reservar hotéis e voos usando a API de Pesquisa do Google da Serpapi.com. Ela constrói os parâmetros necessários, envia solicitações à API e processa as respostas para retornar informações relevantes de reserva. A chave da API (SERPAPI_SEARCH_API_KEY) e o endpoint (SERPAPI_SEARCH_ENDPOINT) são usados para autenticar e enviar solicitações à API de Pesquisa do Google.


In [ ]:
# Define Booking Plugin
class BookingPlugin:
    """Booking Plugin for customers"""
    @kernel_function(description="booking hotel")
    def booking_hotel(self,query: Annotated[str, "The name of the city"], check_in_date: Annotated[str, "Hotel Check-in Time"], check_out_date: Annotated[str, "Hotel Check-in Time"])-> Annotated[str, "Return the result of booking hotel infomation"]:

        params = {
            "engine": "google_hotels",
            "q": query,
            "check_in_date": check_in_date,
            "check_out_date": check_out_date,
            "adults": "2",
            "currency": "USD",
            "gl": "us",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY
        }

        response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=params)
        if response.status_code == 200:
            response = response.json()
            return response["properties"]
        else:
            return None

    
    @kernel_function(description="booking fight")
    def  booking_fight(self,origin: Annotated[str, "The name of Departure"], destination: Annotated[str, "The name of Destination"], outbound_date: Annotated[str, "The date of outbound"], return_date: Annotated[str, "The date of Return_date"])-> Annotated[str, "Return the result of booking fight infomation"]:
        
        go_params = {
            "engine": "google_flights",   
            "departure_id": origin,
            "arrival_id": destination,
            "outbound_date": outbound_date,
            "return_date": return_date,  
            "currency": "USD",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY  
        }

        print(go_params)

        go_response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=go_params)


        result = ''

        if go_response.status_code == 200:
            response = go_response.json()

            result += "# outbound \n " + str(response)
        else:
            print('error!!!')
            # return None

        
        back_params = {
            "engine": "google_flights",   
            "departure_id": destination,
            "arrival_id": origin,
            "outbound_date": return_date,
            "return_date": return_date,  
            "currency": "USD",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY  
        }


        print(back_params)


        back_response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=back_params)



        if back_response.status_code == 200:
            response = back_response.json()

            result += "\n # return \n"  + str(response)

        else:
            print('error!!!')
            # return None
        
        print(result)

        return result

        


# Explicação:
A classe SavePlugin fornece um método saving_plan para salvar planos de viagem utilizando os serviços de Azure AI. Ela configura as credenciais do Azure, cria um agente de IA, processa as entradas do usuário para gerar e salvar o conteúdo do plano de viagem, e gerencia as operações de salvamento de arquivos e limpeza. O método retorna "Saved" após a conclusão bem-sucedida.


In [ ]:
class SavePlugin:
    """Save Plugin for customers"""
    @kernel_function(description="saving plan")
    async def saving_plan(self,tripplan: Annotated[str, "The content of trip plan"])-> Annotated[str, "Return status of save content"]:

        async with (
            DefaultAzureCredential() as creds,
            AzureAIAgent.create_client(
                credential=creds,
                conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
            ) as client,
        ):

            code_interpreter = CodeInterpreterTool()
            
            agent_definition = await client.agents.create_agent(
                model=ai_agent_settings.model_deployment_name,
                tools=code_interpreter.definitions,
                tool_resources=code_interpreter.resources,
            )


            agent = AzureAIAgent(
                client=client,
                definition=agent_definition,
            )

            thread = await client.agents.create_thread()


            user_inputs = [
                """
            
                        You are my Python programming assistant. Generate code,save """+ tripplan +
                        
                    """    
                        and execute it according to the following requirements

                        1. Save blog content to trip-{YYMMDDHHMMSS}.md

                        2. give me the download this file link
                    """
            ]



            try:
                for user_input in user_inputs:
                    # Add the user input as a chat message
                    await agent.add_chat_message(
                        thread_id=thread.id, message=ChatMessageContent(role=AuthorRole.USER, content=user_input)
                    )
                    print(f"# User: '{user_input}'")
                    # Invoke the agent for the specified thread
                    async for content in agent.invoke(thread_id=thread.id):
                        if content.role != AuthorRole.TOOL:
                            print(f"# Agent: {content.content}")

                    
                    messages = await client.agents.list_messages(thread_id=thread.id)

                    # OpenAIPageableListOfThreadMessage
                    # OpenAIPageableListOfThreadMessage


                    for file_path_annotation in messages.file_path_annotations:

                            file_name = os.path.basename(file_path_annotation.text)

                            await client.agents.save_file(file_id=file_path_annotation.file_path.file_id, file_name=file_name,target_dir="./trip")

                    
            finally:
                await client.agents.delete_thread(thread.id)
                await client.agents.delete_agent(agent.id)


        return "Saved"

# Explicação:
Este código configura agentes de IA do Azure para lidar com reservas de voos e hotéis, além de salvar planos de viagem com base nas entradas dos usuários. Ele utiliza credenciais do Azure para criar e configurar os agentes, processa as entradas dos usuários por meio de um chat em grupo e garante a limpeza adequada após a conclusão das tarefas. Os agentes utilizam plugins específicos (BookingPlugin e SavePlugin) para realizar suas respectivas funções.


In [ ]:
async with (
    DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(
        credential=creds,
        conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
    ) as client,
):
    BOOKING_AGENT_NAME = "BookingAgent"
    BOOKING_AGENT_INSTRUCTIONS = """
    You are a booking agent. Help me book flights or hotels.

    Thought: Please understand the user's intention and confirm whether to use the reservation system to complete the task.

    Actions:
    - For flight bookings, convert the departure and destination names into airport codes.
    - Use the appropriate API for hotel or flight bookings. Verify that all necessary parameters are available. If any parameters are missing, ask the user to provide them. If all parameters are complete, call the corresponding function.
    - If the task is not related to hotel or flight booking, respond with the final answer only.
    - Output the results using a markdown table:
      - For flight bookings, output separate outbound and return contents in the order of:
        Departure Airport | Airline | Flight Number | Departure Time | Arrival Airport | Arrival Time | Duration | Airplane | Travel Class | Price (USD) | Legroom | Extensions | Carbon Emissions (kg).
      - For hotel bookings, output in the order of:
        Property Name | Property Description | Check-in Time | Check-out Time | Prices | Nearby Places | Hotel Class | GPS Coordinates.
    """

    SAVE_AGENT_NAME = "SaveAgent"
    SAVE_AGENT_INSTRUCTIONS = """
    You are a save tool agent. Help me to save the trip plan.
    """

    # Create agent definition
    booking_agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=BOOKING_AGENT_NAME,
        instructions=BOOKING_AGENT_INSTRUCTIONS,
    )

    # Create the AzureAI Agent
    booking_agent = AzureAIAgent(
        client=client,
        definition=booking_agent_definition,
        # Optionally configure polling options
        # polling_options=RunPollingOptions(run_polling_interval=timedelta(seconds=1)),
    )

    # Add the sample plugin to the kernel
    booking_agent.kernel.add_plugin(BookingPlugin(), plugin_name="booking")

    # Create agent definition
    save_agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=SAVE_AGENT_NAME,
        instructions=SAVE_AGENT_INSTRUCTIONS
    )

    # Create the AzureAI Agent
    save_agent = AzureAIAgent(
        client=client,
        definition=save_agent_definition,
    )

    save_agent.kernel.add_plugin(SavePlugin(), plugin_name="saving")

    user_inputs = [
        "I have a business trip from London to New York in Feb 20 2025 to Feb 27 2025 ,help me to book a hotel and fight tickets and save it"
    ]

    chat = AgentGroupChat(
        agents=[booking_agent, save_agent],
        termination_strategy=ApprovalTerminationStrategy(agents=[save_agent], maximum_iterations=10),
    )

    try:
        for user_input in user_inputs:
            # Add the user input as a chat message
            await chat.add_chat_message(
                ChatMessageContent(role=AuthorRole.USER, content=user_input)
            )
            print(f"# User: '{user_input}'")

            async for content in chat.invoke():
                print(f"# {content.role} - {content.name or '*'}: '{content.content}'")

            print(f"# IS COMPLETE: {chat.is_complete}")

            print("*" * 60)
            print("Chat History (In Descending Order):\n")
            async for message in chat.get_chat_messages(agent=save_agent):
                print(f"# {message.role} - {message.name or '*'}: '{message.content}'")
    finally:
        await chat.reset()
        await client.agents.delete_agent(save_agent.id)
        await client.agents.delete_agent(booking_agent.id)



---

**Aviso Legal**:  
Este documento foi traduzido utilizando o serviço de tradução por IA [Co-op Translator](https://github.com/Azure/co-op-translator). Embora nos esforcemos para garantir a precisão, esteja ciente de que traduções automáticas podem conter erros ou imprecisões. O documento original em seu idioma nativo deve ser considerado a fonte oficial. Para informações críticas, recomenda-se a tradução profissional feita por humanos. Não nos responsabilizamos por quaisquer mal-entendidos ou interpretações incorretas decorrentes do uso desta tradução.
